In [2]:
import numpy as np
import pandas as pd

In [3]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [3]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA5/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'P2O5', 'l.i.', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'P2O5', 'l.i.', 'oth'],
      dtype='object')

In [4]:
mineralogy = mineralogy.dropna()

In [5]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [6]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["l.i.", "oth"], axis=1))

In [7]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [8]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5
5249,70.057231,0.379448,15.600889,0.840565,2.172153,0.048853,1.030624,2.373740,3.301078,4.151767,0.043653
5250,70.034388,0.380868,15.604671,0.840275,2.179164,0.048881,1.036590,2.380872,3.300390,4.150055,0.043847
5251,70.011515,0.382293,15.608443,0.839983,2.186183,0.048909,1.042588,2.387986,3.299713,4.148337,0.044051
5252,69.988634,0.383723,15.612201,0.839689,2.193203,0.048937,1.048612,2.395074,3.299049,4.146614,0.044264
5253,69.965765,0.385156,15.615943,0.839393,2.200218,0.048966,1.054657,2.402131,3.298397,4.144886,0.044487


In [10]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]
mineralogy3 = mineralogy.iloc[90000:135000]
mineralogy4 = mineralogy.iloc[135000:180000]



In [11]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA5/area5_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA5/area5_2.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA5/area5_3.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy4, "../_CIPW/CIPW/AREA5/area5_4.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [17]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA5/area5_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA5/area5_2_results_excel.txt")
df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA5/area5_3_results_excel.txt")
df_final4 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA5/area5_4_results_excel.txt")

In [21]:
df_final = pd.concat([df_final1, df_final2, df_final3, df_final4])

In [30]:
df_final

,Q,P,K,QAPF
SAMPLE,,,,
5249,27.208198,36.628164,27.326432,monzo granite
5250,27.167389,36.656974,27.315743,monzo granite
5251,27.126472,36.685809,27.305004,monzo granite
5252,27.085567,36.714535,27.294222,monzo granite
5253,27.044643,36.743102,27.283505,monzo granite
...,...,...,...,...
363036,29.914730,34.397391,28.094479,monzo granite
363037,29.915543,34.396695,28.094040,monzo granite
363038,29.916504,34.395953,28.093617,monzo granite


## QAPF classification

In [23]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [24]:
df_final["QAPF"].unique()

array(['monzo granite'], dtype=object)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter_ternary(df_final, a="Q", b="K", c="P", color="QAPF")

fig.write_image(f"../_FIGURES/geomap/qapf_ternary/area5_QAPF.pdf")
fig.show()

In [31]:
df_final.to_excel("../_CIPW/CIPW/AREA5/qapf.xlsx", index=True)

-------

In [25]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA5/QAPF_counts.xlsx")

-----

In [1]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area_subdivided.xlsx", index_col=None, usecols = ['area', 'Y_UTM', 'X_UTM', 'ZoneNumber', 'ZoneLetter'])

NameError: name 'pd' is not defined

In [27]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,-3.582079e+08,-1.175335e+08,1.0,W,1.0,52.549507,17.912818,22.931202,monzo granite
2,-3.561714e+08,-9.094768e+07,1.0,W,1.0,45.940433,19.086384,32.357653,monzo granite
3,-3.549865e+08,-1.354764e+08,1.0,W,1.0,50.739986,6.408381,37.371890,syeno granite
4,-3.398950e+08,-1.053522e+08,1.0,W,1.0,41.407116,23.232361,27.590113,monzo granite
5,-3.525384e+08,-1.868785e+08,1.0,W,1.0,54.633030,17.945563,19.744849,monzo granite
...,...,...,...,...,...,...,...,...,...
4248,5.752663e+05,7.185258e+06,1.0,W,5.0,16.144156,42.276924,25.207393,quartz monzonite
4249,6.042901e+05,7.364584e+06,1.0,W,5.0,20.183025,45.679758,18.892849,granodiorite
4250,4.322847e+05,7.396923e+06,1.0,W,5.0,11.994039,44.790421,23.532209,quartz monzodiorite\nquartz monzogabbro
4251,4.397582e+05,7.402305e+06,1.0,W,5.0,15.151787,43.949506,22.328233,quartz monzodiorite\nquartz monzogabbro


In [28]:
QAPF_control_area5 = QAPF_control[QAPF_control['area'] ==5]
QAPF_control_area5.to_excel("../_CIPW/CIPW/AREA5/QAPF_control_points.xlsx", index=True)

In [29]:
QAPF_control_area2

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
4074,422823.000139,7.525443e+06,1.0,W,5.0,19.204004,43.532090,22.951993,granodiorite
4075,451013.431499,7.286869e+06,1.0,W,5.0,24.588781,25.328080,30.115380,monzo granite
4076,594545.985094,7.366095e+06,1.0,W,5.0,19.011906,34.816617,30.808575,monzo granite
4077,403945.076840,7.527911e+06,1.0,W,5.0,18.198267,43.960602,20.436041,granodiorite
4078,401216.148466,7.529882e+06,1.0,W,5.0,11.072507,45.661909,35.190046,quartz monzonite
...,...,...,...,...,...,...,...,...,...
4248,575266.330715,7.185258e+06,1.0,W,5.0,16.144156,42.276924,25.207393,quartz monzonite
4249,604290.081053,7.364584e+06,1.0,W,5.0,20.183025,45.679758,18.892849,granodiorite
4250,432284.691263,7.396923e+06,1.0,W,5.0,11.994039,44.790421,23.532209,quartz monzodiorite\nquartz monzogabbro
4251,439758.156320,7.402305e+06,1.0,W,5.0,15.151787,43.949506,22.328233,quartz monzodiorite\nquartz monzogabbro
